In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns



In [2]:
q = '''
select *
from workspace.emily_ctr_df
'''
df = pd.read_sql(q, con=bun_dw)

In [3]:
df.columns

Index(['label', 'day', 'hour', 'imp_content_id', 'imp_content_position',
       'imp_page_id', 'imp_ref_page_id', 'imp_ref_term', 'imp_ref_source',
       'imp_user_id', 'imp_content_owner', 'user_sex', 'user_age',
       'user_following_cnt', 'user_bunpay_count', 'owner_grade',
       'owner_item_count', 'owner_interest', 'owner_follower_cnt',
       'owner_bunpay_count', 'content_price', 'content_category_id',
       'content_emergency_cnt', 'content_comment_cnt', 'content_interest',
       'content_pfavcnt'],
      dtype='object')

In [58]:
# df.to_csv('df.csv', index=False)

In [10]:
features = ['hour', 'imp_content_id', 'imp_content_position',
       'imp_page_id', 'imp_ref_page_id', 'imp_ref_term', 'imp_ref_source',
       'imp_user_id', 'imp_content_owner', 'user_sex', 'user_age',
       'user_following_cnt', 'user_bunpay_count', 'owner_grade',
       'owner_item_count', 'owner_interest', 'owner_follower_cnt',
       'owner_bunpay_count', 'content_price', 'content_category_id',
       'content_emergency_cnt', 'content_comment_cnt', 'content_interest',
       'content_pfavcnt']

s = '''
select distinct hour as feature_value,
       'hour' as feature_type
from workspace.emily_ctr_df
'''

for f in features[1:]:
    t = '''
    union
    select distinct {} as feature_value,
           {} as feature_type
    from workspace.emily_ctr_df'''.format(f, '\''+f+'\'')
    s += t

In [11]:
print(s)




select distinct hour as feature_value,
       'hour' as feature_type
from workspace.emily_ctr_df

    union
    select distinct imp_content_id as feature_value,
           'imp_content_id' as feature_type
    from workspace.emily_ctr_df
    union
    select distinct imp_content_position as feature_value,
           'imp_content_position' as feature_type
    from workspace.emily_ctr_df
    union
    select distinct imp_page_id as feature_value,
           'imp_page_id' as feature_type
    from workspace.emily_ctr_df
    union
    select distinct imp_ref_page_id as feature_value,
           'imp_ref_page_id' as feature_type
    from workspace.emily_ctr_df
    union
    select distinct imp_ref_term as feature_value,
           'imp_ref_term' as feature_type
    from workspace.emily_ctr_df
    union
    select distinct imp_ref_source as feature_value,
           'imp_ref_source' as feature_type
    from workspace.emily_ctr_df
    union
    select distinct imp_user_id as feature_value,
   

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

sparse_features = ['hour', 'imp_content_id', 'imp_content_position', 'imp_page_id', 'imp_ref_page_id', 'imp_ref_term', 'imp_ref_source', 'imp_user_id', 'imp_content_owner', 'user_sex', 'content_category_id']
for f in sparse_features:
    df[f] = df[f].fillna('')
    df[f] = le.fit_transform(df[f])

In [5]:
df.head()

,label,day,hour,imp_content_id,imp_content_position,imp_page_id,imp_ref_page_id,imp_ref_term,imp_ref_source,imp_user_id,...,owner_item_count,owner_interest,owner_follower_cnt,owner_bunpay_count,content_price,content_category_id,content_emergency_cnt,content_comment_cnt,content_interest,content_pfavcnt
0,0,20200601,1,14344,2808,1,0,13769,1,5761,...,941,106,0,0,23000,57,0,0,268,12
1,0,20200601,1,8103,3652,1,0,9593,1,9506,...,1541,25831,0,179,31000,31,1,0,800,19
2,0,20200601,1,7643,1699,1,0,9339,0,11148,...,154,198,0,0,15000,8,1,1,4198,148
3,0,20200601,1,11990,2809,1,0,6007,1,15590,...,1013,12202,0,22,13500,8,0,0,3929,448
4,0,20200601,1,10351,1699,1,0,6007,1,14626,...,461,178535,0,4,12000,58,0,0,1636,107


In [65]:
print(df.columns)

Index(['label', 'day', 'hour', 'imp_content_id', 'imp_content_position',
       'imp_page_id', 'imp_ref_page_id', 'imp_ref_term', 'imp_ref_source',
       'imp_user_id', 'imp_content_owner', 'user_sex', 'user_age',
       'user_following_cnt', 'user_bunpay_count', 'owner_grade',
       'owner_item_count', 'owner_interest', 'owner_follower_cnt',
       'owner_bunpay_count', 'content_price', 'content_category_id',
       'content_emergency_cnt', 'content_comment_cnt', 'content_interest',
       'content_pfavcnt'],
      dtype='object')


In [66]:
print(sparse_features)

['hour', 'imp_content_id', 'imp_content_position', 'imp_page_id', 'imp_ref_page_id', 'imp_ref_term', 'imp_ref_source', 'imp_user_id', 'imp_content_owner', 'user_sex', 'content_category_id']


In [64]:
dense_features = [i for i in df.columns if i not in sparse_features]
print(dense_features)

['label', 'day', 'user_age', 'user_following_cnt', 'user_bunpay_count', 'owner_grade', 'owner_item_count', 'owner_interest', 'owner_follower_cnt', 'owner_bunpay_count', 'content_price', 'content_emergency_cnt', 'content_comment_cnt', 'content_interest', 'content_pfavcnt']


In [15]:
import optuna
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_auc_score

In [12]:
from sklearn.model_selection import train_test_split


x = df.loc[:, df.columns != 'label']
y = df.loc[:, df.columns == 'label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [50]:
df.columns

Index(['label', 'day', 'hour', 'imp_content_id', 'imp_content_position',
       'imp_page_id', 'imp_ref_page_id', 'imp_ref_term', 'imp_ref_source',
       'imp_user_id', 'imp_content_owner', 'user_sex', 'user_age',
       'user_following_cnt', 'user_bunpay_count', 'owner_grade',
       'owner_item_count', 'owner_interest', 'owner_follower_cnt',
       'owner_bunpay_count', 'content_price', 'content_category_id',
       'content_emergency_cnt', 'content_comment_cnt', 'content_interest',
       'content_pfavcnt'],
      dtype='object')

In [13]:
print('size of x train:', len(x_train))
print('label 1 percentage of train set: ', round(len(y_train[y_train['label'] == 1])/len(y_train)*100, 2), '%')
print('label 1 percentage of test set: ', round(len(y_test[y_test['label'] == 1])/len(y_test)*100, 2), '%')

size of x train: 453611
label 1 percentage of train set:  2.92 %
label 1 percentage of test set:  2.96 %


In [16]:
# class_weight: dict or ‘balanced’, default=None
lr = LogisticRegression(class_weight=None, n_jobs=-1, random_state=42)
lr.fit(x_train, y_train)
pred = lr.predict(x_test)

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [19]:
from sklearn.metrics import log_loss

print('log loss:', log_loss(y_test.astype(int), pred.astype(int)))

log loss: 1.0212121130140792


In [84]:
coef = lr.coef_[0].tolist()
coef_dict = {}
for i in range(len(coef)):    
    print('Feature [', x_train.columns[i], f'], Score: {coef[i]}')
    coef_dict[x_train.columns[i]] = [coef[i]]

Feature [ day ], Score: -1.6876684677430814e-07
Feature [ hour ], Score: 6.5404918596586305e-09
Feature [ imp_content_id ], Score: 8.265324859108812e-06
Feature [ imp_content_position ], Score: -5.238034040927616e-05
Feature [ imp_page_id ], Score: -2.4268562237544064e-08
Feature [ imp_ref_page_id ], Score: 1.6363372998860752e-09
Feature [ imp_ref_term ], Score: -2.3580231438973585e-05
Feature [ imp_ref_source ], Score: -1.1374358929457116e-08
Feature [ imp_user_id ], Score: 2.509059255667858e-07
Feature [ imp_content_owner ], Score: -2.131773175516209e-06
Feature [ user_sex ], Score: 1.099010090224094e-08
Feature [ user_age ], Score: -3.481353089072639e-08
Feature [ user_following_cnt ], Score: -1.1177256064544116e-06
Feature [ user_bunpay_count ], Score: 2.0310165837546576e-08
Feature [ owner_grade ], Score: 2.458105943661038e-06
Feature [ owner_item_count ], Score: -1.923078677977168e-05
Feature [ owner_interest ], Score: 1.075403951986582e-07
Feature [ owner_follower_cnt ], Score: 

In [87]:
coeff = pd.DataFrame.from_dict(coef_dict).T
coeff.columns = ['coefficient']
coeff.sort_values('coefficient', ascending=False)

,coefficient
imp_content_id,8.265325e-06
owner_grade,2.458106e-06
content_pfavcnt,2.385022e-06
content_category_id,8.924017e-07
imp_user_id,2.509059e-07
owner_interest,1.075404e-07
content_price,4.539871e-08
user_bunpay_count,2.031017e-08
content_emergency_cnt,1.432957e-08
user_sex,1.099010e-08


In [53]:
wo_ids = df.drop(['imp_content_id', 'imp_content_owner', 'imp_user_id'], axis=1)
X = wo_ids.loc[:, wo_ids.columns != 'label']
Y = wo_ids.loc[:, wo_ids.columns == 'label']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=0)

In [88]:
lr = LogisticRegression(class_weight=None, n_jobs=-1, random_state=42)
lr.fit(X_train, Y_train)
pred = lr.predict(X_test)
print('log loss:', log_loss(Y_test.astype(int), pred.astype(int)))

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



log loss: 1.0162020364545297


In [90]:
coef = lr.coef_[0].tolist()
coef_dict = {}
for i in range(len(coef)):    
    coef_dict[X_train.columns[i]] = [coef[i]]
coeff = pd.DataFrame.from_dict(coef_dict).T
coeff.columns = ['coefficient']
coeff.sort_values('coefficient', ascending=False)

Feature [ day ], Score: -1.6309552622088938e-07
Feature [ hour ], Score: 1.0814622259251604e-08
Feature [ imp_content_position ], Score: -8.0679543123818e-05
Feature [ imp_page_id ], Score: -4.324107535299572e-08
Feature [ imp_ref_page_id ], Score: 2.5936912822602622e-09
Feature [ imp_ref_term ], Score: -1.1456918023718572e-05
Feature [ imp_ref_source ], Score: -2.000159086345854e-08
Feature [ user_sex ], Score: 1.8079908030260514e-08
Feature [ user_age ], Score: -6.997027201670438e-08
Feature [ user_following_cnt ], Score: -1.9080407427651744e-06
Feature [ user_bunpay_count ], Score: 3.10705792896247e-08
Feature [ owner_grade ], Score: -2.0889662916709073e-06
Feature [ owner_item_count ], Score: -2.7468945261887907e-05
Feature [ owner_interest ], Score: 2.590866811076594e-07
Feature [ owner_follower_cnt ], Score: 0.0
Feature [ owner_bunpay_count ], Score: -1.1463652404917933e-06
Feature [ content_price ], Score: 4.642307578696436e-08
Feature [ content_category_id ], Score: 1.481612031

,coefficient
content_pfavcnt,3.875651e-06
content_category_id,1.481612e-06
owner_interest,2.590867e-07
content_price,4.642308e-08
user_bunpay_count,3.107058e-08
content_emergency_cnt,2.353235e-08
user_sex,1.807991e-08
hour,1.081462e-08
imp_ref_page_id,2.593691e-09
owner_follower_cnt,0.000000e+00


In [97]:
df_small = df[coeff.sort_values('coefficient', ascending=False).index[:10].tolist() + ['label']]
X = df_small.loc[:, df_small.columns != 'label']
Y = df_small.loc[:, df_small.columns == 'label']
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2, random_state = 0) 

lr_small = LogisticRegression(n_jobs=-1, random_state=42)
lr_small.fit(trainX, trainY)
p = lr_small.predict(testX)
print('log loss:', log_loss(testY, p))

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



log loss: 1.0291014252475303


In [20]:
lr = LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=42)
lr.fit(x_train, y_train)
pred = lr.predict(x_test)
print('log loss:', log_loss(y_test.astype(int), pred.astype(int)))

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



log loss: 16.61418248377593


In [22]:
print(len([i for i in pred if i == 1])/len(y_test))

0.4842376303977849


In [25]:
import random
import math
lr_class_weight = random.randint(2, 10)
print(lr_class_weight)
lr = LogisticRegression(class_weight={0: 1, 1: lr_class_weight}, n_jobs=-1, random_state=42)
lr.fit(x_train, y_train)
pred = lr.predict(x_test)
print('log loss:', log_loss(y_test.astype(int), pred.astype(int)))

2


/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



log loss: 1.0212121130140792


In [26]:
[i for i in pred if i == 1]

[]

In [27]:
rf_class_weight = int(math.log(random.randint(20, 100)))
rf_n_estimators = int(math.log(random.randint(20, 70)))
rf_max_depth = int(math.log(random.randint(3, 7)))

print('rf_class_weight:', rf_class_weight)
print('rf_n_estimators:', rf_n_estimators)
print('rf_max_depth:', rf_max_depth)
rf = RandomForestClassifier(n_estimators=rf_n_estimators,
                                        max_depth=rf_max_depth,
                                        class_weight={0: 1, 1: rf_class_weight},
                                        n_jobs=-1,
                                        random_state=42)
rf.fit(x_train, y_train)
pred = rf.predict(x_test)
print('log loss:', log_loss(y_test.astype(int), pred.astype(int)))

rf_class_weight: 4
rf_n_estimators: 3
rf_max_depth: 1


/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:13: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



log loss: 1.0212121130140792


In [29]:
rf_class_weight = random.randint(20, 100)
rf_n_estimators = random.randint(20, 70)
rf_max_depth = random.randint(3, 7)
print('rf_class_weight:', rf_class_weight)
print('rf_n_estimators:', rf_n_estimators)
print('rf_max_depth:', rf_max_depth)
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
rf.fit(x_train, y_train)
pred = rf.predict(x_test)
print('log loss:', log_loss(y_test.astype(int), pred.astype(int)))

rf_class_weight: 47
rf_n_estimators: 33
rf_max_depth: 4


/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



log loss: 1.044666871742417


In [31]:
len([i for i in pred if i == 1])/len(pred)

0.00713385007451302

In [32]:
rf_class_weight = random.randint(20, 100)
rf_n_estimators = random.randint(20, 70)
rf_max_depth = random.randint(3, 7)

print('rf_class_weight:', rf_class_weight)
print('rf_n_estimators:', rf_n_estimators)
print('rf_max_depth:', rf_max_depth)
rf = RandomForestClassifier(n_estimators=rf_n_estimators,
                                        max_depth=rf_max_depth,
                                        class_weight={0: 1, 1: rf_class_weight},
                                        n_jobs=-1,
                                        random_state=42)
rf.fit(x_train, y_train)
pred = rf.predict(x_test)
print('log loss:', log_loss(y_test.astype(int), pred.astype(int)))

rf_class_weight: 98
rf_n_estimators: 49
rf_max_depth: 7


/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:13: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



log loss: 32.40116359202549


In [33]:
len([i for i in pred if i == 1])/len(pred)

0.9672495436628661

In [34]:
xgb_class_weight = int(math.log(random.randint(20, 100)))
xgb_n_estimators = int(math.log(random.randint(20, 70)))
xgb_max_depth = int(math.log(random.randint(3, 7)))
xgb = XGBClassifier(n_estimators=xgb_n_estimators,
                               max_depth=xgb_max_depth,
                               objective='binary:logistic',
                               scale_pos_weight=xgb_class_weight,
                               n_jobs=-1,
                               random_state=42)
xgb.fit(x_train.iloc[:, 1:], y_train)
pred = xgb.predict(x_test.iloc[:, 1:])
print('log loss:', log_loss(y_test.astype(int), pred.astype(int)))

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



log loss: 1.0212121130140792


In [35]:
len([i for i in pred if i == 1])/len(pred)

0.0

In [36]:
[i for i in pred if i == 1]

[]